## Analysis of PyCity Schools

* Charter schools performed better across all metrics when compared to district schools. However, the average number of students per charter schools is approximately 60% less than the average number of students per district schools. Additional analysis is required to determine school practices given the much lower average student population per charter school.

* Overall, schools with lower per-student budgets performed better than schools with higher per-student budgets. Additionally, schools with small to medium populations (up to 2,500) performed better than schools with large populations. Considering though that charter schools have, on average, smaller student populations, further analysis is needed to determine if these results are skewed.

In [20]:
# setup dependencies
import pandas as me

# load files
school_data_load = "Resources/schools_complete.csv"
student_data_load = "Resources/students_complete.csv"

# read school and student data files to store into DataFrames
school_data = me.read_csv(school_data_load)
student_data = me.read_csv(student_data_load)

# combine seperate data files into single dataset
school_data_combined = me.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

### District Summary

In [21]:
# GET DISTRICT METRICS
# schools and students totals
passMin = 70   # variable for min passing grade, can change if needed
countSchools = len(school_data_combined["school_name"].unique())     # count unique school names
countStudent = school_data_combined["Student ID"].count()            # count student IDs

# calculate total budget
totalBudget = school_data["budget"].sum()

# calculate average scores
mathAvg = school_data_combined["math_score"].mean()
readAvg = school_data_combined["reading_score"].mean()

# percent pass rates calculations
passMathCount = school_data_combined[(school_data_combined["math_score"]) >= passMin].count()["student_name"]    
passReadCount = school_data_combined[(school_data_combined["reading_score"]) >= passMin].count()["student_name"]  
passOvrCount = school_data_combined[(school_data_combined["math_score"] >= passMin) 
                                    & (school_data_combined["reading_score"] >= passMin)].count()["student_name"]
    
passMathPct = passMathCount / float(countStudent) * 100
passReadPct = passReadCount / float(countStudent) * 100
passOvrPct = passOvrCount / float(countStudent) * 100

In [22]:
# new datatframe for district summary
districtSummary = me.DataFrame({"Total Schools":  [countSchools],
                                "Total Students": [countStudent],
                                "Total Budget": [totalBudget],
                                "Average Math Score": [mathAvg],
                                "Average Reading Score": [readAvg],
                                "% Passing Math": [passMathPct],
                                "% Passing Reading": [passReadPct],
                                "% Overall Passing": [passOvrPct]})

districtSummary = districtSummary[["Total Schools", "Total Students", "Total Budget", "Average Math Score", 
                                    "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]

# formatting
districtSummary["Total Students"] = districtSummary["Total Students"].map("{:,}".format)
districtSummary["Total Budget"] = districtSummary["Total Budget"].map("${:,.2f}".format)

# change display on floating values, show 3 decimals (cleaner)
me.set_option('display.precision', 3)

# display the dataframe
districtSummary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985,81.878,74.981,85.805,65.172


### School Summary

In [23]:
# get school type
schoolType = school_data.set_index(["school_name"])["type"]

# get total student count per school
perSchoolCount = school_data_combined["school_name"].value_counts()

# get total school budget and per-student spending
perSchoolBudget = school_data_combined.groupby(["school_name"]).mean()["budget"]
perSchoolCapita = perSchoolBudget / perSchoolCount

# get average test scores
perSchoolMath = school_data_combined.groupby(["school_name"]).mean()["math_score"]
perSchoolRead = school_data_combined.groupby(["school_name"]).mean()["reading_score"]

In [24]:
# create new dataframes to allow metric splits by school for math/reading/overall pass counts
schoolPassMath = school_data_combined[(school_data_combined["math_score"] >= passMin)]
schoolPassRead = school_data_combined[(school_data_combined["reading_score"] >= passMin)]
schoolPassBoth = school_data_combined[(school_data_combined["math_score"] >= passMin) &
                                      (school_data_combined["reading_score"] >= passMin)]

In [25]:
# percentage pass rates
schoolPassMath = schoolPassMath.groupby(["school_name"]).count()["student_name"] / perSchoolCount * 100
schoolPassRead = schoolPassRead.groupby(["school_name"]).count()["student_name"] / perSchoolCount * 100
schoolPassBoth = schoolPassBoth.groupby(["school_name"]).count()["student_name"] / perSchoolCount * 100

In [26]:
# new datatframe for school summary
perSchoolSummary = me.DataFrame({"School Type": schoolType,
                                 "Total Students": perSchoolCount,
                                 "Total School Budget": perSchoolBudget,
                                 "Per-Student Budget": perSchoolCapita,
                                 "Average Math Score": perSchoolMath,
                                 "Average Reading Score": perSchoolRead,
                                 "% Passing Math": schoolPassMath,
                                 "% Passing Reading": schoolPassRead,
                                 "% Overall Passing": schoolPassBoth})

perSchoolSummary = perSchoolSummary[["School Type", "Total Students", "Total School Budget", "Per-Student Budget", 
                                     "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", 
                                     "% Overall Passing"]]

# formatting
perSchoolSummary["Total School Budget"] = perSchoolSummary["Total School Budget"].map("${:,.2f}".format)
perSchoolSummary["Per-Student Budget"] = perSchoolSummary["Per-Student Budget"].map("${:,.2f}".format)

# display the dataframe
perSchoolSummary

,School Type,Total Students,Total School Budget,Per-Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048,81.034,66.680,81.933,54.642
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.062,83.976,94.133,97.040,91.335
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.712,81.158,65.988,80.739,53.204
Ford High School,District,2739,"$1,763,916.00",$644.00,77.103,80.746,68.310,79.299,54.290
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351,83.817,93.392,97.139,90.599
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.290,80.934,66.753,80.863,53.528
Holden High School,Charter,427,"$248,087.00",$581.00,83.803,83.815,92.506,96.253,89.227
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629,81.183,65.684,81.316,53.514
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072,80.966,66.058,81.222,53.539
Pena High School,Charter,962,"$585,858.00",$609.00,83.840,84.045,94.595,95.946,90.541


### Top Performing Schools by Overall Passing %

In [27]:
# sort and show top 5
topSchools = perSchoolSummary.sort_values(["% Overall Passing"], ascending=False)
topSchools.head(5)

,School Type,Total Students,Total School Budget,Per-Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.062,83.976,94.133,97.040,91.335
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418,83.849,93.272,97.309,90.948
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351,83.817,93.392,97.139,90.599
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274,83.989,93.868,96.540,90.583
Pena High School,Charter,962,"$585,858.00",$609.00,83.840,84.045,94.595,95.946,90.541


### Bottom Performing Schools by Overall Passing %

In [28]:
# sort and show bottom 5
bottomSchools = perSchoolSummary.sort_values(["% Overall Passing"], ascending=True)
bottomSchools.head(5)

,School Type,Total Students,Total School Budget,Per-Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.843,80.745,66.367,80.220,52.988
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.712,81.158,65.988,80.739,53.204
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629,81.183,65.684,81.316,53.514
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.290,80.934,66.753,80.863,53.528
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072,80.966,66.058,81.222,53.539


### Math Scores by Grade

In [29]:
# data series of math scores by grade level
ninthGrade = school_data_combined[(school_data_combined["grade"] == "9th")]
tenthGrade = school_data_combined[(school_data_combined["grade"] == "10th")]
eleventhGrade = school_data_combined[(school_data_combined["grade"] == "11th")]
twelfthGrade = school_data_combined[(school_data_combined["grade"] == "12th")]

# group by school name and format to 3 decimal places (cleaner)
ninthMathScores = ninthGrade.groupby(["school_name"]).mean()["math_score"]
tenthMathScores = tenthGrade.groupby(["school_name"]).mean()["math_score"]
eleventhMathScores = eleventhGrade.groupby(["school_name"]).mean()["math_score"]
twelfthMathScores = twelfthGrade.groupby(["school_name"]).mean()["math_score"]

# combine to single dataframe
gradeMathScores = me.DataFrame({"9th": ninthMathScores, "10th": tenthMathScores,
                                "11th": eleventhMathScores, "12th": twelfthMathScores})

gradeMathScores = gradeMathScores[["9th", "10th", "11th", "12th"]]
gradeMathScores.index.name = None

# display dataframe
gradeMathScores

,9th,10th,11th,12th
Bailey High School,77.084,76.997,77.516,76.492
Cabrera High School,83.095,83.155,82.766,83.277
Figueroa High School,76.403,76.540,76.884,77.151
Ford High School,77.361,77.672,76.918,76.180
Griffin High School,82.044,84.229,83.842,83.356
Hernandez High School,77.438,77.337,77.136,77.187
Holden High School,83.787,83.430,85.000,82.855
Huang High School,77.027,75.909,76.447,77.226
Johnson High School,77.188,76.691,77.492,76.863
Pena High School,83.625,83.372,84.328,84.122


### Reading Scores by Grade

In [30]:
# data series of reading scores by grade level
ninthGrade = school_data_combined[(school_data_combined["grade"] == "9th")]
tenthGrade = school_data_combined[(school_data_combined["grade"] == "10th")]
eleventhGrade = school_data_combined[(school_data_combined["grade"] == "11th")]
twelfthGrade = school_data_combined[(school_data_combined["grade"] == "12th")]

# group by school name and format to 3 decimal places (cleaner)
ninthReadingScores = ninthGrade.groupby(["school_name"]).mean()["reading_score"]
tenthReadingScores = tenthGrade.groupby(["school_name"]).mean()["reading_score"]
eleventhReadingScores = eleventhGrade.groupby(["school_name"]).mean()["reading_score"]
twelfthReadingScores = twelfthGrade.groupby(["school_name"]).mean()["reading_score"]

# combine to single dataframe
gradeReadingScores = me.DataFrame({"9th": ninthReadingScores, "10th": tenthReadingScores,
                                "11th": eleventhReadingScores, "12th": twelfthReadingScores})

gradeReadingScores = gradeReadingScores[["9th", "10th", "11th", "12th"]]
gradeReadingScores.index.name = None

# display dataframe
gradeReadingScores

,9th,10th,11th,12th
Bailey High School,81.303,80.907,80.946,80.912
Cabrera High School,83.676,84.253,83.788,84.288
Figueroa High School,81.199,81.409,80.640,81.385
Ford High School,80.633,81.263,80.404,80.662
Griffin High School,83.369,83.707,84.288,84.014
Hernandez High School,80.867,80.660,81.396,80.857
Holden High School,83.677,83.325,83.816,84.699
Huang High School,81.290,81.512,81.417,80.306
Johnson High School,81.261,80.773,80.616,81.228
Pena High School,83.807,83.612,84.336,84.591


### Scores by School Spending

In [31]:
# establish bins
spendingBins = [0, 585, 630, 645, 680]
binNames = ["<$585", "$585-630", "$630-645", "$645-680"]

# create copy of school summary
schoolSpending = perSchoolSummary

# categorize spending based on established bins
schoolSpending["Spending Ranges Per Student"] = me.cut(perSchoolCapita, spendingBins, labels=binNames, right=False)
schoolSpending

,School Type,Total Students,Total School Budget,Per-Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges Per Student
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048,81.034,66.680,81.933,54.642,$585-630
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.062,83.976,94.133,97.040,91.335,<$585
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.712,81.158,65.988,80.739,53.204,$630-645
Ford High School,District,2739,"$1,763,916.00",$644.00,77.103,80.746,68.310,79.299,54.290,$630-645
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351,83.817,93.392,97.139,90.599,$585-630
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.290,80.934,66.753,80.863,53.528,$645-680
Holden High School,Charter,427,"$248,087.00",$581.00,83.803,83.815,92.506,96.253,89.227,<$585
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629,81.183,65.684,81.316,53.514,$645-680
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072,80.966,66.058,81.222,53.539,$645-680
Pena High School,Charter,962,"$585,858.00",$609.00,83.840,84.045,94.595,95.946,90.541,$585-630


In [32]:
# get averaages for each spending category
spendingMathScores = schoolSpending.groupby(["Spending Ranges Per Student"]).mean()["Average Math Score"]
spendingReadScores = schoolSpending.groupby(["Spending Ranges Per Student"]).mean()["Average Reading Score"]
spendingPassMath = schoolSpending.groupby(["Spending Ranges Per Student"]).mean()["% Passing Math"]
spendingPassRead = schoolSpending.groupby(["Spending Ranges Per Student"]).mean()["% Passing Reading"]
spendingPassBoth = schoolSpending.groupby(["Spending Ranges Per Student"]).mean()["% Overall Passing"]

In [33]:
# create dataframe
spendingSummary = me.DataFrame({"Average Math Score": spendingMathScores,
                                "Average Reading Score": spendingReadScores,
                                "% Passing Math": spendingPassMath,
                                "% Passing Reading": spendingPassRead,
                                "% Overall Passing": spendingPassBoth})

spendingSummary = spendingSummary[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading",
                                   "% Overall Passing"]]

# display results
spendingSummary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges Per Student,,,,,
<$585,83.455,83.934,93.460,96.611,90.369
$585-630,81.900,83.155,87.134,92.718,81.419
$630-645,78.519,81.624,73.484,84.392,62.858
$645-680,76.997,81.028,66.165,81.134,53.527


### Scores by School Size

In [34]:
# establish bins
sizeBins = [0, 1000, 2500, 5000]
binNames = ["Small (<1000)", "Medium (1000-2500)", "Large (2500-5000)"]

# create copy of school summary
schoolSize = perSchoolSummary

# categorize spending based on established bins
schoolSize["School Size"] = me.cut(schoolSize["Total Students"], sizeBins, labels=binNames, right=False)
schoolSize                                

,School Type,Total Students,Total School Budget,Per-Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges Per Student,School Size
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048,81.034,66.680,81.933,54.642,$585-630,Large (2500-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.062,83.976,94.133,97.040,91.335,<$585,Medium (1000-2500)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.712,81.158,65.988,80.739,53.204,$630-645,Large (2500-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.103,80.746,68.310,79.299,54.290,$630-645,Large (2500-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351,83.817,93.392,97.139,90.599,$585-630,Medium (1000-2500)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.290,80.934,66.753,80.863,53.528,$645-680,Large (2500-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803,83.815,92.506,96.253,89.227,<$585,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629,81.183,65.684,81.316,53.514,$645-680,Large (2500-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072,80.966,66.058,81.222,53.539,$645-680,Large (2500-5000)
Pena High School,Charter,962,"$585,858.00",$609.00,83.840,84.045,94.595,95.946,90.541,$585-630,Small (<1000)


In [35]:
# get averaages for each size category
sizeMathScores = schoolSize.groupby(["School Size"]).mean()["Average Math Score"]
sizeReadScores = schoolSize.groupby(["School Size"]).mean()["Average Reading Score"]
sizePassMath = schoolSize.groupby(["School Size"]).mean()["% Passing Math"]
sizePassRead = schoolSize.groupby(["School Size"]).mean()["% Passing Reading"]
sizePassBoth = schoolSize.groupby(["School Size"]).mean()["% Overall Passing"]

In [36]:
# create dataframe
sizeSummary = me.DataFrame({"Average Math Score": sizeMathScores,
                            "Average Reading Score": sizeReadScores,
                            "% Passing Math": sizePassMath,
                            "% Passing Reading": sizePassRead,
                            "% Overall Passing": sizePassBoth})

sizeSummary = sizeSummary[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading",
                           "% Overall Passing"]]

# display results
sizeSummary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.822,83.930,93.550,96.099,89.884
Medium (1000-2500),83.358,83.885,93.644,96.749,90.615
Large (2500-5000),76.957,80.967,66.548,80.799,53.672


### Scores by School Type

In [37]:
# new series grouping by school type
typeAvgStudents = perSchoolSummary.groupby(["School Type"]).mean()["Total Students"].round(1)
typeMathScores = perSchoolSummary.groupby(["School Type"]).mean()["Average Math Score"]
typeReadScores = perSchoolSummary.groupby(["School Type"]).mean()["Average Reading Score"]
typePassMath = perSchoolSummary.groupby(["School Type"]).mean()["% Passing Math"]
typePassRead = perSchoolSummary.groupby(["School Type"]).mean()["% Passing Reading"]
typePassBoth = perSchoolSummary.groupby(["School Type"]).mean()["% Overall Passing"]

In [38]:
# create dataframe
typeSummary = me.DataFrame({"Average # of Students": typeAvgStudents,
                            "Average Math Score" : typeMathScores,
                            "Average Reading Score": typeReadScores,
                            "% Passing Math": typePassMath,
                            "% Passing Reading": typePassRead,
                            "% Overall Passing": typePassBoth})

typeSummary = typeSummary[["Average # of Students", "Average Math Score", "Average Reading Score", "% Passing Math", 
                           "% Passing Reading", "% Overall Passing"]]

# Display results
typeSummary

,Average # of Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,,
Charter,1524.2,83.474,83.896,93.621,96.586,90.432
District,3853.7,76.957,80.967,66.548,80.799,53.672
